In [7]:
import numpy as np
import pandas as pd
from tslearn.piecewise import PiecewiseAggregateApproximation
from tqdm import tqdm
from sklearn import preprocessing
from pyts.classification import KNeighborsClassifier
from sklearn.metrics.pairwise import manhattan_distances
from sklearn.gaussian_process import GaussianProcessClassifier
from pyts.transformation import BOSS
from pyts.classification import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.ensemble import VotingClassifier
from pyts.transformation import WEASEL
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import FunctionTransformer
import xgboost as xgb
from sklearn.metrics import accuracy_score

le = preprocessing.LabelEncoder()
PATH = "G:/Coding/ML/UCRArchive_2018/"
dataset_list = ['Adiac',
'Beef',
'Car',
'CBF',
'ECG200',
'FacesUCR',
'GunPoint',
'Ham',
'Herring',
'Meat',
'MoteStrain',
'OliveOil',
'Plane',
'Strawberry',
'SyntheticControl',
'TwoLeadECG',
'Wine',
'UMD',
'Coffee',
'CricketX',
'CricketY',
'CricketZ',
'DiatomSizeReduction',
'FaceAll',
'FaceFour',
'ItalyPowerDemand',
'Lightning2',
'Lightning7','MedicalImages']
warping_window_list = [3,
0,
1,
11,
0,
12,
0,
0,
5,
0,
1,
0,
5,
0,
6,
4,
0,
4,
3,
7,
17,
7,
0,
3,
2,
0,
6,
5,
20
]
window_size_list = [32,24,16,16,40,32,32,32,32,32,16,32,16,16,40,16,16,16,16,80,112,128,32,40,64,16,128,128,64]
word_size_list= [8,4,8,4,8,8,4,4,4,4,8,4,4,4,8,8,8,8,4,4,8,4,4,8,8,4,8,4,4]
number_of_bins_list = [9,7,9,3,4,3,9,9,9,9,6,7,7,7,7,7,3,8,8,8,5,6,3,5,3,3,3,4,4,]

In [8]:
ALL = [['Dataset','MD_XGBOOST','DTWR_XGBOOST','DTW_XGBOOST','SAXDist_XGBOOST','SAX_XGBOOST','DTW_DTWR_SAX_SAXD_ED_XGBOOST','DTW_DTWR_ED_MD_XGBOOST','DTW_DTWR_SAX_XGBOOST','PAADist_XGBOOST','PAA_XGBOOST','DTW_DTWR_XGBOOST']]

In [ ]:
for i,(dataset,warping_window,window_size,word_size,number_of_bins) in tqdm(enumerate(zip(dataset_list,warping_window_list,window_size_list,word_size_list,number_of_bins_list))):
    app = []
    app.append(dataset)
    file_train = PATH + str(dataset) + "/" + str(dataset) + "_TRAIN.tsv"
    file_test = PATH + str(dataset) + "/" + str(dataset) + "_TEST.tsv"

    fulltrain = np.genfromtxt(fname=file_train, delimiter="\t", skip_header=0)
    fulltest = np.genfromtxt(fname=file_test, delimiter="\t", skip_header=0)

    X_train, y_train = fulltrain[:,1:],fulltrain[:,0]
    X_test, y_test = fulltest[:,1:],fulltest[:,0]
    
    y_test = le.fit_transform(y_test)
    y_train = le.fit_transform(y_train)
    y_train=pd.DataFrame(y_train)
    y_test=pd.DataFrame(y_test)
    
    y_train = y_train[0]
    y_test = y_test[0]

    numc = []
    for i in range(len(y_test.value_counts())):
        numc.append(y_test.value_counts().index[i])
    number_of_classes = len(numc)
    
    params = {
    'max_depth': 2,
    'objective': 'multi:softmax',  # error evaluation for multiclass training
    'num_class': int(number_of_classes),
    'n_gpus': 0
    }
    #Feature Extraction
    mandist_train = manhattan_distances(X_train)
    mandist_test = manhattan_distances(X_test,X_train)

    EDist_test = []
    for i in range(len(y_test)):
        for j in range(len(y_train)):
            dist = np.sqrt(np.sum((np.array(X_test[i,:])-np.array(X_train[j,:]))**2))
            EDist_test.append(dist)

    EDist_train = []
    for i in range(len(y_train)):
        for j in range(len(y_train)):
            dist1 = np.sqrt(np.sum((np.array(X_train[i,:])-np.array(X_train[j,:]))**2))
            EDist_train.append(dist1)

    EDist_train = np.array(EDist_train)
    EDist_train.resize(X_train.shape[0],X_train.shape[0])
    EDist_test = np.array(EDist_test)
    EDist_test.resize(X_test.shape[0],X_train.shape[0])

    from pyts.metrics import dtw

    DTW_Classic_test = []
    DTW_Classic_train = []
    for i in range(len(X_test)):
        for j in range(len(X_train)):
            dtw_classic, path_classic = dtw(X_test[i], X_train[j], dist='square',
                                    method='classic', return_path=True)
            DTW_Classic_test.append(dtw_classic)

    for i in range(len(X_train)):
        for j in range(len(X_train)):
            dtw_classic, path_classic = dtw(X_train[i], X_train[j], dist='square',method='classic', return_path=True)
            DTW_Classic_train.append(dtw_classic)
    DTW_Classic_train = np.array(DTW_Classic_train)
    DTW_Classic_train.resize(X_train.shape[0],X_train.shape[0])
    DTW_Classic_test = np.array(DTW_Classic_test)
    DTW_Classic_test.resize(X_test.shape[0],X_train.shape[0])


    #DTWR
    DTW_sakoe_test = []
    DTW_sakoe_train = []
    for i in range(len(X_test)):
        for j in range(len(X_train)):
            dtw_sakoechiba, path_sakoechiba = dtw(X_test[i], X_train[j], dist='square', method='sakoechiba',options={'window_size': warping_window}, return_path=True)
            DTW_sakoe_test.append(dtw_sakoechiba)

    for i in range(len(X_train)):
        for j in range(len(X_train)):
            dtw_sakoechiba, path_sakoechiba = dtw(X_train[i], X_train[j], dist='square', method='sakoechiba',options={'window_size': warping_window}, return_path=True)
            DTW_sakoe_train.append(dtw_sakoechiba)

    DTW_sakoe_train = np.array(DTW_sakoe_train)
    DTW_sakoe_train.resize(X_train.shape[0],X_train.shape[0])
    DTW_sakoe_test = np.array(DTW_sakoe_test)
    DTW_sakoe_test.resize(X_test.shape[0],X_train.shape[0])

    from tslearn.piecewise import SymbolicAggregateApproximation, OneD_SymbolicAggregateApproximation
    from tslearn.piecewise import PiecewiseAggregateApproximation


    sax = SymbolicAggregateApproximation(n_segments=window_size, alphabet_size_avg=word_size)
    Xtrain_sax = sax.inverse_transform(sax.fit_transform(X_train))
    Xtest_sax = sax.inverse_transform(sax.fit_transform(X_test))

    Xtest_sax =Xtest_sax[:,:,0]
    Xtrain_sax = Xtrain_sax[:,:,0]

    #SAX Transform + SAX feature extraction


    SAXDist_test = []
    for i in range(len(y_test)):
        for j in range(len(y_train)):
            #dtw_sakoechiba, path_sakoechiba = dtw(SAX_test[i], SAX_train[j], dist='square', method='sakoechiba',options={'window_size': window_size}, return_path=True)
            dtw_classic, path_classic = dtw(Xtest_sax[i], Xtrain_sax[j], dist='square',method='classic', return_path=True)
            #dist = np.sqrt(np.sum((np.array(SAX_test[i,:])-np.array(SAX_train[j,:]))**2))
            #SAXDist_test.append(dtw_sakoechiba)
            SAXDist_test.append(dtw_classic)
            #SAXDist_test.append(dist)

    SAXDist_train = []
    for i in range(len(y_train)):
        for j in range(len(y_train)):
            #dtw_sakoechiba, path_sakoechiba = dtw(SAX_train[i], SAX_train[j], dist='square', method='sakoechiba',options={'window_size': window_size}, return_path=True)
            dtw_classic, path_classic = dtw(Xtrain_sax[i], Xtrain_sax[j], dist='square',method='classic', return_path=True)
            #dist1 = np.sqrt(np.sum((np.array(SAX_train[i,:])-np.array(SAX_train[j,:]))**2))
            #SAXDist_train.append(dtw_sakoechiba)
            SAXDist_train.append(dtw_classic)
            #SAXDist_train.append(dist1)

    SAXDist_train = np.array(SAXDist_train)
    SAXDist_train.resize(X_train.shape[0],X_train.shape[0])
    SAXDist_test = np.array(SAXDist_test)
    SAXDist_test.resize(X_test.shape[0],X_train.shape[0])

    paa = PiecewiseAggregateApproximation(n_segments=window_size)
    PAA_train = paa.fit_transform(X_train)
    PAA_test = paa.fit_transform(X_test)
    PAA_train = PAA_train[:,:,0]
    PAA_test = PAA_test[:,:,0]
    
    PAA_dist_test = []
    for i in range(len(y_test)):
        for j in range(len(y_train)):
            PAA_dists = paa.distance_paa(PAA_test[i,:],PAA_train[j,:])
            PAA_dist_test.append(PAA_dists)

    PAA_dist_train = []
    for i in range(len(y_train)):
        for j in range(len(y_train)):
            PAA_dists = paa.distance_paa(PAA_train[i,:],PAA_train[j,:])
            PAA_dist_train.append(PAA_dists)

    PAA_dist_train = np.array(PAA_dist_train)
    PAA_dist_train.resize(X_train.shape[0],X_train.shape[0])

    PAA_dist_test = np.array(PAA_dist_test)
    PAA_dist_test.resize(X_test.shape[0],X_train.shape[0])
    
    
    
    test_DTW_DTWR = np.hstack((DTW_sakoe_test,DTW_Classic_test))
    train_DTW_DTWR = np.hstack((DTW_sakoe_train,DTW_Classic_train))

    test_DTW_DTWR_SAX = np.hstack((DTW_sakoe_test,DTW_Classic_test,Xtest_sax))
    train_DTW_DTWR_SAX = np.hstack((DTW_sakoe_train,DTW_Classic_train,Xtrain_sax))

    test_DTW_DTWR_ED_MD = np.hstack((DTW_sakoe_test,DTW_Classic_test,EDist_test,mandist_test))
    train_DTW_DTWR_ED_MD = np.hstack((DTW_sakoe_train,DTW_Classic_train,EDist_train,mandist_train))

    test = np.hstack((DTW_sakoe_test,DTW_Classic_test,SAXDist_test,Xtest_sax,EDist_test))
    train = np.hstack((DTW_sakoe_train,DTW_Classic_train,SAXDist_train,Xtrain_sax,EDist_train))

    
    
    mandist_train=pd.DataFrame(mandist_train)
    mandist_test=pd.DataFrame(mandist_test)

    DTW_sakoe_train=pd.DataFrame(DTW_sakoe_train)
    DTW_sakoe_test=pd.DataFrame(DTW_sakoe_test)

    DTW_Classic_train=pd.DataFrame(DTW_Classic_train)
    DTW_Classic_test=pd.DataFrame(DTW_Classic_test)

    SAXDist_test=pd.DataFrame(SAXDist_test)
    SAXDist_train=pd.DataFrame(SAXDist_train)

    Xtest_sax=pd.DataFrame(Xtest_sax)
    Xtrain_sax=pd.DataFrame(Xtrain_sax)

    test=pd.DataFrame(test)
    train=pd.DataFrame(train)
    
    train_DTW_DTWR_ED_MD=pd.DataFrame(train_DTW_DTWR_ED_MD)
    test_DTW_DTWR_ED_MD=pd.DataFrame(test_DTW_DTWR_ED_MD)
    
    test_DTW_DTWR_SAX=pd.DataFrame(test_DTW_DTWR_SAX)
    train_DTW_DTWR_SAX=pd.DataFrame(train_DTW_DTWR_SAX)
    
    train_DTW_DTWR=pd.DataFrame(train_DTW_DTWR)
    test_DTW_DTWR=pd.DataFrame(test_DTW_DTWR)
    
    PAA_dist_test=pd.DataFrame(PAA_dist_test)
    PAA_dist_train=pd.DataFrame(PAA_dist_train)
    
    PAA_train=pd.DataFrame(PAA_train)
    PAA_test=pd.DataFrame(PAA_test)
    

    dtrain_md = xgb.DMatrix(data=mandist_train, label=y_train)
    dtest_md = xgb.DMatrix(data=mandist_test)
    bst_md = xgb.train(params, dtrain_md)
    pred_md = bst_md.predict(dtest_md)
    acc_md = accuracy_score(y_test, pred_md)
    app.append(1-acc_md)
    
    dtrain_DTWR = xgb.DMatrix(data=DTW_sakoe_train, label=y_train)
    dtest_DTWR = xgb.DMatrix(data=DTW_sakoe_test)
    bst_DTWR = xgb.train(params, dtrain_DTWR)
    pred_DTWR = bst_DTWR.predict(dtest_DTWR)
    acc_DTWR = accuracy_score(y_test, pred_DTWR)
    app.append(1-acc_DTWR)
    
    dtrain_DTW = xgb.DMatrix(data=DTW_Classic_train, label=y_train)
    dtest_DTW = xgb.DMatrix(data=DTW_Classic_test)
    bst_DTW = xgb.train(params, dtrain_DTW)
    pred_DTW = bst_DTW.predict(dtest_DTW)
    acc_DTW = accuracy_score(y_test, pred_DTW)
    app.append(1-acc_DTW)
    
    dtrain_SAXDIST = xgb.DMatrix(data=SAXDist_train, label=y_train)
    dtest_SAXDIST = xgb.DMatrix(data=SAXDist_test)
    bst_SAXDIST = xgb.train(params, dtrain_SAXDIST)
    pred_SAXDIST = bst_SAXDIST.predict(dtest_SAXDIST)
    acc_SAXDIST = accuracy_score(y_test, pred_SAXDIST)
    app.append(1-acc_SAXDIST)
    
    dtrain_SAX = xgb.DMatrix(data=Xtrain_sax, label=y_train)
    dtest_SAX = xgb.DMatrix(data=Xtest_sax)
    bst_SAX = xgb.train(params, dtrain_SAX)
    pred_SAX = bst_SAX.predict(dtest_SAX)
    acc_SAX = accuracy_score(y_test, pred_SAX)
    app.append(1-acc_SAX)

    dtrain_DTW_DTWR_SAX_SAXD_ED = xgb.DMatrix(data=train, label=y_train)
    dtest_DTW_DTWR_SAX_SAXD_ED = xgb.DMatrix(data=test)
    bst_DTW_DTWR_SAX_SAXD_ED = xgb.train(params, dtrain_DTW_DTWR_SAX_SAXD_ED)
    pred_DTW_DTWR_SAX_SAXD_ED = bst_DTW_DTWR_SAX_SAXD_ED.predict(dtest_DTW_DTWR_SAX_SAXD_ED)
    acc_DTW_DTWR_SAX_SAXD_ED = accuracy_score(y_test, pred_DTW_DTWR_SAX_SAXD_ED)
    app.append(1-acc_DTW_DTWR_SAX_SAXD_ED)

    dtrain_DTW_DTWR_ED_MD = xgb.DMatrix(data=train_DTW_DTWR_ED_MD, label=y_train)
    dtest_DTW_DTWR_ED_MD = xgb.DMatrix(data=test_DTW_DTWR_ED_MD)
    bst_DTW_DTWR_ED_MD = xgb.train(params, dtrain_DTW_DTWR_ED_MD)
    pred_DTW_DTWR_ED_MD = bst_DTW_DTWR_ED_MD.predict(dtest_DTW_DTWR_ED_MD)
    acc_DTW_DTWR_ED_MD = accuracy_score(y_test, pred_DTW_DTWR_ED_MD)
    app.append(1-acc_DTW_DTWR_ED_MD)
    
    dtrain_DTW_DTWR_SAX = xgb.DMatrix(data=train_DTW_DTWR_SAX, label=y_train)
    dtest_DTW_DTWR_SAX = xgb.DMatrix(data=test_DTW_DTWR_SAX)
    bst_DTW_DTWR_SAX = xgb.train(params, dtrain_DTW_DTWR_SAX)
    pred_DTW_DTWR_SAX = bst_DTW_DTWR_SAX.predict(dtest_DTW_DTWR_SAX)
    acc_DTW_DTWR_SAX = accuracy_score(y_test, pred_DTW_DTWR_SAX)
    app.append(1-acc_DTW_DTWR_SAX)
    
    dtrain_PAADIST = xgb.DMatrix(data=PAA_dist_train, label=y_train)
    dtest_PAADIST = xgb.DMatrix(data=PAA_dist_test)
    bst_PAADIST = xgb.train(params, dtrain_PAADIST)
    pred_PAADIST = bst_PAADIST.predict(dtest_PAADIST)
    acc_PAADIST = accuracy_score(y_test, pred_PAADIST)
    app.append(1-acc_PAADIST)
    
    dtrain_PAA = xgb.DMatrix(data=PAA_train, label=y_train)
    dtest_PAA = xgb.DMatrix(data=PAA_test)
    bst_PAA = xgb.train(params, dtrain_PAA)
    pred_PAA = bst_PAA.predict(dtest_PAA)
    acc_PAA = accuracy_score(y_test, pred_PAA)
    app.append(1-acc_PAA)
    
    dtrain_DTW_DTWR = xgb.DMatrix(data=train_DTW_DTWR, label=y_train)
    dtest_DTW_DTWR = xgb.DMatrix(data=test_DTW_DTWR)
    bst_DTW_DTWR = xgb.train(params, dtrain_DTW_DTWR)
    pred_DTW_DTWR = bst_DTW_DTWR.predict(dtest_DTW_DTWR)
    acc_DTW_DTWR = accuracy_score(y_test, pred_DTW_DTWR)
    app.append(1-acc_DTW_DTWR)
    
    
    ALL.append(app)

import pandas as pd
df = pd.DataFrame(ALL)
export_csv = df.to_csv('G:/Coding/ML/Feature_resultsALL/results_XGBOOST_21june_1_'+str(len(ALL)-1)+'_csv.csv', index=None, header=True)


0it [00:00, ?it/s]C:\Users\Ishan Yash\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\Ishan Yash\Anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \

1it [07:54, 474.43s/it]C:\Users\Ishan Yash\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\Ishan Yash\Anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \

2it [08:15, 338.41s/it]C:\Users\Ishan Yash\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and


19it [50:30, 79.70s/it] C:\Users\Ishan Yash\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\Ishan Yash\Anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \

20it [1:04:24, 306.15s/it]

In [ ]:
export_csv = df.to_csv('G:/Coding/ML/Feature_resultsALL/results_xgb_22june_1_'+str(len(ALL)-1)+'_csv.csv', index=None, header=True)